# Working with Raw Gaze Samples

Once gaze data have been loaded, they are available as time-ordered raw samples in `gaze.samples`. 
The term "raw gaze data" is used inconsistently in the literature. It may refer to:

- Original eye-tracker files containing mixed content (samples, events, messages)
- Gaze coordinates over time without filtering or event classification
- Vendor-provided event labels (e.g., fixations or saccades)

In `pymovements`, "raw samples" refer specifically to the lowest-level gaze time series available after import, before smoothing, velocity computation, or event detection. All higher-level measures are derived from these samples.

The table below shows a simplified example of raw gaze samples after import ({py:class}`~pymovements.Gaze`). Each row corresponds to one time-ordered gaze sample and is stored in the ``samples`` attribute.

In [ ]:
import matplotlib.pyplot as plt

import pymovements as pm
from pymovements.gaze.experiment import Experiment

experiment = Experiment(
    screen_width_px=1680,
    screen_height_px=1050,
    screen_width_cm=47.5,
    screen_height_cm=30,
    distance_cm=65,
    origin='upper left',
    sampling_rate=1000.0,
)

gaze_0 = pm.gaze.from_csv(
    "../examples/gaze-toy-example.csv",
    experiment=experiment,
    time_column="timestamp",
)

gaze_0.samples.head(8)

Column dtypes after import:

- ``time`` (``i64``): timestamp of the sample, typically in milliseconds.
- ``x`` (``f64``): horizontal gaze position in pixel coordinates.
- ``y`` (``f64``): vertical gaze position in pixel coordinates.
- `stimuli_x` and `stimuli_y` describe the position of the currently presented stimulus on the screen at each time point. These values are typically given in pixel coordinates and allow gaze samples to be related directly to the visual content shown to the participant. If no stimulus position was available at a given time, these fields may contain placeholder values (for example `-1`) indicating the absence of an active stimulus.

Depending on the loader and input format, additional channels such as binocular coordinates or quality measures may also be present.

In [ ]:
gaze = pm.gaze.from_csv(
    "../examples/gaze-toy-example.csv",
    experiment=experiment,
    time_column="timestamp",
    pixel_columns=['x', 'y']
)

# drop columns stimuli_x and stimuli_y
gaze.samples = gaze.samples.drop(['stimuli_x', 'stimuli_y'])
gaze.samples.head(8)

## Inspecting Raw Samples with Plots

Visual inspection is an essential first step when working with newly loaded gaze data. Time-series plots help reveal signal loss, noise, blinks, sampling irregularities, or calibration problems before any preprocessing is applied. 

Using the {py:func}`~pymovements.plotting.traceplot` function, we can visualize raw gaze samples from a {py:class}`~pymovements.Gaze` object. The plot shows the continuous trajectory of gaze positions across the stimulus, allowing inspection of spatial gaze behavior over time. 

In [ ]:
pm.plotting.traceplot(gaze)

plt.show()

The {py:func}`~pymovements.plotting.tsplot` function produces a time-series plot which shows how each recorded signal changes over time, with one line per selected channel (e.g., horizontal and vertical gaze position). The x-axis represents time, as defined by the gaze sample timestamps.

In this example, we plot the `x` and `y` channels to examine the raw gaze signal before applying any event detection or preprocessing. 

In [ ]:
pm.plotting.tsplot(
    gaze_0,
    channels=['x', 'y'],
    share_y=False,
    line_color="darkblue",
    zero_centered_yaxis=False)

plt.show()

See the {doc}`Plotting Gaze Data tutorial <../tutorials/plotting>` for an example of time-series visualization using ``traceplot``.

## Transforming Raw Samples

Raw pixel coordinates are tied to a specific screen setup and viewing distance. For meaningful interpretation and cross-experiment comparison, gaze samples are often transformed into alternative representations. These transformations operate directly on the raw samples and rely on the experimental metadata defined earlier.

### From Pixels to Degrees of Visual Angle

Eye trackers typically record gaze positions in screen pixels. While useful for display-based inspection, pixel units depend on screen size and viewing distance and are therefore not comparable across setups. The function {py:func}`~pymovements.gaze.transforms.pix2deg` converts pixel coordinates into degrees of visual angle (dva) using the experiment's screen geometry and viewing distance.

Requirements:
- A pixel-based gaze column must be available (by default named "pixel")
- An {py:class}`~pymovements.Experiment` must be attached to the gaze data, because screen size and distance are needed for the conversion

In [ ]:
gaze.pix2deg()

gaze

### From Position to Velocity

Many eye-movement measures are derived not from position directly but from its temporal
derivatives. Velocity is computed from changes in gaze position over time and is central to event detection algorithms for saccades and fixations. In pymovements, velocity is computed explicitly from position data with the {py:func}`~pymovements.gaze.transforms.pos2vel` function, using the sampling rate stored in the eye tracker definition.

In [ ]:
gaze.pos2vel()

In [ ]:
gaze